In [1]:
import os
import re
import warnings

import pandas as pd
import numpy as np
import torch

from transformers import (
    AutoConfig, AutoTokenizer, 
    AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, 
    AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
)

from datasets import Dataset

from src.metrics import rouge_for_batch, f1_score_at_k_for_batch, jaccard_similarity_for_batch

os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings('ignore')

In [2]:
NGPU = torch.cuda.device_count()
NCPU = os.cpu_count()
NGPU, NCPU

(6, 64)

# Paths and Names

In [3]:
### paths and names

DATA_PATH = 'data/model_dev/model_dev_v3.pickle'
MODEL_CHECKPOINT = '.log/paust_pko_t5_base_v3_run_5/checkpoint-11310'

# Model & Tokenizer

In [4]:
config = AutoConfig.from_pretrained(MODEL_CHECKPOINT)

In [5]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT, config=config)
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

# Training Args

In [6]:
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1

predict_with_generate=True
generation_max_length=64

# Functions

In [7]:
prefix = "generate keyphrases: "

max_input_length = 1024
max_target_length = 64

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["input_text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")

    labels = tokenizer(examples["target_text"], max_length=max_target_length, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [8]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    
    # Replace -100 in the labels as we can't decode them.
    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # ROUGE / Mean Generated Length
    result = rouge_for_batch(decoded_labels, decoded_preds)
    result["gen_len"] = np.mean(prediction_lens)

    # F1@K / Jaccard@K
    result["F1@10"] = f1_score_at_k_for_batch(decoded_labels, decoded_preds, 10)
    result["jaccard_similarity"] = jaccard_similarity_for_batch(decoded_labels, decoded_preds, 10)
    
    result = {key: round(value * 100, 4) for key, value in result.items()}
    
    return result

# Inputs and Labels

In [9]:
data_df = pd.read_pickle(DATA_PATH)

In [10]:
dataset = Dataset.from_pandas(data_df).shuffle(seed=100).train_test_split(0.2, seed=100)
train_dataset = dataset['train']
eval_dataset = dataset['test']

In [11]:
train_dataset = train_dataset.map(preprocess_function, 
                                  batched=True, 
                                  num_proc=NCPU, 
                                  remove_columns=train_dataset.column_names)

eval_dataset = eval_dataset.map(preprocess_function, 
                                batched=True, 
                                num_proc=NCPU, 
                                remove_columns=eval_dataset.column_names)
print(train_dataset)
print(eval_dataset)

Map:   0%|          | 0/9346 [00:00<?, ? examples/s]

Map:   0%|          | 0/2337 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 9346
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2337
})


In [12]:
tokenizer.decode(train_dataset['input_ids'][0])

'generate keyphrases: "로페테기가 좋아하는 황희찬, 리즈전 선발 가능성 있어" 복귀전에서 득점을 한 황희찬은 선발 출전을 대기하고 있다.울버햄튼은 19일 오전 0시(한국시간) 영국 울버햄튼에 위치한 몰리뉴 스타디움에서 리즈 유나이티드와 2022-23시즌 잉글리시 프리미어리그(EPL) 28라운드를 치른다. 울버햄튼은 13위, 리즈는 19위에 위치 중이다.영국 \'익스프레스 앤 스타\'는 울버햄튼, 리즈 경기를 프리뷰하면서 "홈 팬들은 시작부터 공격적이고 강렬한 경기를 원한다. 훌렌 로페테기 감독은 때로는 보수적인 방식으로 접근을 했다. 홈 팬들 앞에서는 달라야 한다. 공격진 변화가 예상되는 황희찬이 선발로 나설 수 있다"고 전했다.황희찬은 브루노 라즈 감독 아래에선 벤치 신세였다. 로페테기 감독이 온 후엔 달랐다. 득점 수는 적어도 기동력과 저돌적인 황희찬을 선호했다. 경기력으로 응답했다. 2022 국제축구연맹(FIFA) 카타르 월드컵에서 활약으로 자신감까지 올랐다. 좋은 활약을 이어가던 황희찬은 리버풀전에서 부상을 당해 한동안 빠졌다.재활 기간을 거친 황희찬은 지난 뉴캐슬 유나이티드전에 복귀를 했는데 골을 넣었다. 0-1로 뒤지던 후반 24분 들어와 후반 25분 집중력 있는 모습으로 뉴캐슬 골망을 흔들었다. 경기는 울버햄튼의 1-2 패배로 끝이 났지만 황희찬에겐 고무적인 날이었다. 이날 골로 황희찬은 리그 1호 득점에 성공했고 EPL에선 무려 13개월 만에 골 맛을 봤다. 부상 불운을 골로 보답을 받는 날이었다.\'익스프레스 앤 스타\'는 "로페테기 감독은 황희찬을 정말 좋아한다. 근면한 모습과 라울 히메네스를 대체할 수 있는 능력이 있어 리즈전에 선발로 택할 수 있다. 지난 주말 햄스트링 부상에서 돌아온 황희찬은 뉴캐슬을 상대로 골을 넣었다. 황희찬이 선발로 나오면 레프트백 라얀 아이트-누리와 같이 가능성이 높다"고 구체적인 전망을 내놓았다.울버햄튼 순위는 13위지만 강등권인 18위 본머스와 승점 3점차밖에 안 난다. 리즈전 필승이 요구되는

In [13]:
tokenizer.decode(train_dataset['labels'][0])

'로페테기; 황희찬; 선발 출전; 울버햄튼; 리즈전; EPL; 득점; 복귀전; 한동안; 부상</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

# Trainer for Generation

In [14]:
training_args = Seq2SeqTrainingArguments(
    output_dir='./.temp',
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,

    predict_with_generate=predict_with_generate,
    generation_max_length=generation_max_length,
)

In [15]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True)

trainer = Seq2SeqTrainer(
    model=model,
    
    args=training_args,
    
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    
    tokenizer=tokenizer,
    data_collator=data_collator,

    compute_metrics=compute_metrics,
)

# Generate

In [16]:
preds = trainer.predict(eval_dataset)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [17]:
preds.metrics

{'test_loss': 8.006477355957031,
 'test_rouge1': 65.7355,
 'test_rouge2': 45.4681,
 'test_rougeL': 54.0561,
 'test_rougeLsum': 54.0561,
 'test_gen_len': 5009.4138,
 'test_F1@10': 59.7773,
 'test_jaccard_similarity': 26.0938,
 'test_runtime': 724.7455,
 'test_samples_per_second': 3.225,
 'test_steps_per_second': 0.135}

In [18]:
for data, pred in zip(eval_dataset, preds.predictions):
    pred = np.where(pred != -100, pred, tokenizer.pad_token_id)
    # context = tokenizer.decode(data['input_ids'], skip_special_tokens=True)
    summary = tokenizer.decode(data['labels'], skip_special_tokens=True)
    pred = tokenizer.decode(pred, skip_special_tokens=True)
    # print(f'입력: {context}')
    print(f'정답: {summary}')
    print(f'예측: {pred}', end='\n\n')

정답: 토트넘; 우승; 아스널; 맨시티; 승점 차; 빅4; 브라이튼; 강등 전쟁; 슈퍼컴퓨터; 노팅엄 포레스트
예측: 토트넘; 빅4; 슈퍼컴퓨터; 우승; 브라이턴; 뉴캐슬; 맨체스터; 승점; 강등 전쟁; 레스터시티

정답: 경부고속도; 지하화; 서울 리니어파크; 강남 도심; 상부 공간; 녹지 공간; 연결; 서울시장; 지하도로; 착공
예측: 경부고속도로; 양재~한남; 지하화; 상부공간; 이용방안; 서울 리니어파크; 강남도심; 동서 지역; 리오 공원; 복합문화 공간

정답: 중기부; 중소기업부; 수출활성화; TF(태스크포스); 글로벌 강소기업; 육성; 1000+ 프로젝트; 글로벌 비즈니스센터(GBC); 개편전략; 수출인큐베이터(BI)
예측: 중소벤처기업부; 수출 활성화; 민관 협·단체장; 연구기관; 합동 대응체계; 수출바우처; 글로벌 비즈니스센터; 수출바우처; 수출바우처 지원; 수출 선도기업; 스마트공장 우대지원

정답: 하태경; 윤석열 대통령; 부산세계박람회; BIE; 일광수산 횟집; 친일몰이; 더탐사; 선라이즈; 조선시대 지명; 괴담 언론
예측: 하태경; 윤석열 대통령; 부산세계박람회; 부산 횟집; 친일몰이; 더탐사; 일광; 욱일기 상징; 일제강점기; 조선시대 지명

정답: SSG 추신수; 외야 수비; 1번 우익수; 대전 한화전; 첫 수비 출장; 지명타자; 개막 7경기; 왼쪽 팔꿈치 수술; 스프링캠프; 선발 라인업
예측: SSG 추신수; 외야 수비; 추신수; 선발 라인업; 1번타자 우익수; 개막 7경기; 첫 수비; 주전 우익수; 베이스러닝; 선발 라인업; 선발투수

정답: 전처 남친 살해; 재판; 혐의; 징역 19년; B씨; 이혼; 주거지; 1심; C씨; 대법원
예측: 이혼한 전 부인; 남자친구; 살해; 징역 19년; 전처 남친; 대법원; 살인 혐의; C씨; 흉기로 살해; 원심 확정; 양형

정답: 은행권 발행; 코코본드; 조건부자본증권; 발행 잔액; 상각; 회계상 자본; 상각 사유; 자기자본비율; 투자심리 위축; 금융시장
예측: 국내 은행권; 코코본드; 상각